<a><img src="https://cdn.iconscout.com/icon/free/png-256/data-science-46-1170621.png" width="200" align="center"></a>
<h1><center>Applied Data Science Capstone</center></h1>

<h3><center>Segmenting and Clustering Neighborhoods in Toronto</center></h3>
<h4><center>Part 1 - Scraping & Importing the Data</center></h4>

Welcome to my week 3 notebook. In this assignment, I will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

<h2>Import Data</hh2>

<h4>Import Libraries</h4>

In [2]:
#Install dependencies if needed
#!pip install bs4
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup
import re

<h4>Create SSL</h4>

In [3]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

<h4>Set Up Handler</h4>

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
fhand = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(fhand, 'html.parser')

<h4>Populate Table</h4>

In [5]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df = df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df.shape

(288, 3)

<h4>Drop All Rows with 'Not assigned'</h4>

In [7]:
df = df[~df['Borough'].isin(['Not assigned'])]

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
df.shape

(211, 3)

<h4>Reset Index</h4>

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<h4>Concatenate Neighborhoods & Fix Buroughs</h4>

In [12]:
def concatenate_neighbourhood(x):
    conc = ""
    for i in range(len(x)-1):
        conc = conc + x.iloc[i] + ", "
    conc += x.iloc[-1]
    return conc

def fix_borough(x):
    ref = x.iloc[0]
    for i in range(1, len(x)):
        if ref != x.iloc[i]:
            for i in x:
                print(x)
    return ref

In [13]:
df = df.groupby(["Postcode"]).agg({"Borough": lambda x: fix_borough(x), 
                                   "Neighbourhood": lambda x: concatenate_neighbourhood(x)})
df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [14]:
rows = df.shape[0]
print('There are %d rows in the dataframe' % rows)

There are 103 rows in the dataframe


In [15]:
df.to_csv('ny-neighborhoods.csv')